<a href="https://colab.research.google.com/github/jburgy/blog/blob/master/notebooks/DataFrame%20Formatting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

df = pd.DataFrame(np.transpose([
    [0.330, 4.87, 5.97, 0.073, 0.642, 1898, 568, 86.8, 102, 0.0146],
    [4879, 12104, 12756, 3475, 6792, 142984, 120536, 51118, 49528, 2370],
    [5427, 5243, 5514, 3340, 3933, 1326, 687, 1271, 1638, 2095],
    [3.7, 8.9, 9.8, 1.6, 3.7, 23.1, 9.0, 8.7, 11.0, 0.7],
    [4222.6, 2802.0, 24.0, 708.7, 24.7, 9.9, 10.7, 17.2, 16.1, 153.3],
    [57.9, 108.2, 149.6, 0.384, 227.9, 778.6, 1433.5, 2872.5, 4495.1, 5906.4],
    [167, 464, 15, -20, -65, -110, -140, -195, -200, -225],
    [0, 0, 1, 0, 2, 79, 82, 27, 14, 5],
]), index=pd.MultiIndex.from_tuples([
    ("Terrestial", "Mercury"),
    ("Terrestial", "Venus"),
    ("Terrestial", "Earth"),
    ("Terrestial", "Moon"),
    ("Terrestial", "Mars"),
    ("Jovian", "Jupiter"),
    ("Jovian", "Saturn"),
    ("Jovian", "Uranus"),
    ("Jovian", "Neptune"),
    ("Dwarf", "Pluto"),
], names=["Type", "Name"]), columns=[
    "Mass (10<sup>24</sup>kg)",
    "Diameter (km)",
    "Density (kg/m<sup>3</sup>)",
    "Gravity (m/s<sup>2</sup>)",
    "Length of day (hours)",
    "Distance from Sun (10<sup>6</sup>km)",
    "Mean temperature (°C)",
    "Number of moons",
])

In [0]:
from matplotlib import cm, colors
from pandas.io.formats.style import Styler
from typing import Optional

cm.register_cmap(
    name="RdWhGn", 
    cmap=colors.LinearSegmentedColormap.from_list(
        "RdWhGn",
        ["#d65f5f", "#ffffff", "#5fba7d"],
    ))

EPSILON = np.finfo(float).eps


class Tsyler(Styler):
    @staticmethod
    def _background_gradient(
        s,
        cmap="PuBu",
        low=0,
        high=0,
        text_color_threshold=0.408,
        vmin:Optional[float] = None,
        vmax:Optional[float] = None,
    ):
        vmin, vmax = s.min(), s.max()
        norm = colors.DivergingNorm(0, vmin=min(-EPSILON, vmin), vmax=max(EPSILON, vmax))
        rgbas = cm.get_cmap(cmap)(norm(s.values))
        luminance = np.where(
            rgbas <= 0.03928,
            rgbas / 12.92,
            (rgbas + 0.055) / 1.055 ** 2.4,
        ) @ np.r_[0.2125, 0.7152, 0.0722, 0]
        background = np.apply_along_axis(colors.to_hex, 1, rgbas)
        return np.where(
            luminance < text_color_threshold,
            np.char.mod("background-color: %s; color: #f1f1f1;", background),
            np.char.mod("background-color: %s; color: #000000;", background)
        )

In [0]:
style = Tsyler(
    df,
    table_attributes='border="1" class="dataframe"',
).format("{:0,.0f}").background_gradient(cmap="RdWhGn")

In [0]:
from IPython import display
from IPython.core.magic import Magics, cell_magic, magics_class
import jinja2

@magics_class
class JinjaMagics(Magics):
    """Magics class containing the jinja2 magic and state"""

    def __init__(self, shell):
        super().__init__(shell)
        self.env = jinja2.Environment(loader=jinja2.FileSystemLoader('.'))
        self.display_functions = dict(
            html=display.HTML,
            latex=display.Latex,
            json=display.JSON,
            pretty=display.Pretty,
            display=display.display,
        )
    
    @cell_magic
    def jinja(self, line, cell):
        f = self.display_functions.get(line.lower().strip(), display.display)
        tmp = self.env.from_string(cell)
        rend = tmp.render({
            k: v for k, v in self.shell.user_ns.items()
            if not k.startswith('_') and k not in self.shell.user_ns_hidden
        })
        return f(rend)

get_ipython().register_magics(JinjaMagics)

In [22]:
# based on https://stackoverflow.com/a/17557830
%%jinja html
<style>
/* important styles */

.container {
   /* Attach fixed-th-table to this container,
      in order to layout fixed-th-table
      in the same way as scolled-td-table" */
   position: relative;
    
   /* Truncate fixed-th-table */
   overflow: hidden;
}

.fixed-th-table-wrapper td,
.fixed-th-table-wrapper th,
.scrolled-td-table-wrapper td,
.scrolled-td-table-wrapper th {
   /* Set background to non-transparent color
      because two tables are one above another.
    */
   background: lightgrey;
}
.fixed-th-table-wrapper {
   /* Make table out of flow */
   position: absolute;
}
.fixed-th-table-wrapper tbody th,
.fixed-th-table-wrapper thead th.index_name {
    /* Place fixed-th-table th-cells above 
       scrolled-td-table td-cells.
     */
    position: relative;
    z-index: 1;
}
.scrolled-td-table-wrapper td,
.scrolled-td-table-wrapper thead th:not(.index_name) {
    /* Place scrolled-td-table td-cells
       above fixed-th-table.
     */
    position: relative;
}
.scrolled-td-table-wrapper {
   /* Make horizonal scrollbar if needed */
   overflow-x: auto;
}


/* Simulating border-collapse: collapse,
   because fixed-th-table borders
   are below ".scrolling-td-wrapper table" borders
*/

table {
    border-spacing: 0;
}
td, th {
   border-style: solid;
   border-color: black;
   border-width: 1px 1px 0 0;
}
th:first-child {
   border-left-width: 1px;
}
tr:last-child td,
tr:last-child th {
   border-bottom-width: 1px;
}

/* Unimportant styles */

.container {
    width: 500px;
}
td, th {
   padding: 5px;
}
</style>
<div class="container">
<div class="fixed-th-table-wrapper">
{{style._repr_html_() | safe}}
</div>
<div class="scrolled-td-table-wrapper">
{{style._repr_html_() | safe}}
</div>
</div>